Recommendation system
=

> ## 목차
> ---
> ### 1. 목표 및 의의   
> ### 2. 이론 [go](#3-이론)
>   > 1 Filtering       
>   > 2 Matrix Factorization 
>
> ### 3. 코드 분석 [go](#4-코드-분석)
>   > [0 코드 흐름](#0-코드-흐름)   
>   > [1 데이터 준비](#1-데이터-준비)    
>   > [2 추천시스템 만들기](#2-추천시스템-만들기)   
>   > [3 모델 평가](#3-모델-평가)   
> ### 4. 아쉬운 점 [go](#5-아쉬운-점)

<br><br><br>


- - - -

# 1. 목표 및 의의

### 1) csr Matrix에 대해서 이해하고 활용한다.

### 2) 추천시스템의 원리를 이해하고, 성과를 얻어본다.

<br><br><br>

- - - - 
# 2. 이론

### 1) Filtering
  - Collaborative filtering
 소비자의 구매 습성을 분석해서, 소비자 군을 구분한다고 생각할 수 있다. 콘텐츠를 구분하지 않아도 추천할 수 있다는 장점을 가지고 있다.  

  - Content-based filtering
 콘텐츠 기반의 추천이다. 새로 나온 상품도 해당 상품의 정보만 있으면, 바로 추천시스템에 적용할 수 있어진다.



### 2) Model-based Collaborative Filtering algorithm: 모델 기반 협력 필터링
 

 - Matrix Fatorization 

 필터링을 위해서는 결국 방대한 양의 소비 정보가 필요합니다. 이를 위해, 아마존은 사용자의 검색내역, 구매내역을 사용하였고, 다양한 플렛폼에서 동일한 일을하려고, 소비자들의 데이터를 수집하고 있습니다.

 CSR matrix 로 용량을 줄여줄 필요가 있습니다.
 ![image](data/matrix_factorization.png)


 이번 프로젝트에서는 Matrix Factorization 모델을 implicit 패키지를 사용하여 학습해보겠습니다.

 implicit 패키지는 이전 스텝에서 설명한 암묵적(implicit) dataset을 사용하는 다양한 모델을 굉장히 빠르게 학습할 수 있는 패키지입니다.
 이 패키지에 구현된 als(AlternatingLeastSquares) 모델을 사용하겠습니다. Matrix Factorization에서 쪼개진 두 Feature Matrix를 한꺼번에 훈련하는 것은 잘 수렴하지 않기 때문에, 한쪽을 고정시키고 다른 쪽을 학습하는 방식을 번갈아 수행하는 AlternatingLeastSquares 방식이 효과적인 것으로 알려져 있습니다.   
 [implicit패키지](https://github.com/benfred/implicit)



 이러한 필터링에도 문제는 있습니다. 다음으로는 추천시스템의 한계, 현 시점에 대해서 논해보도록 하겠습니다. (본 프로젝트로만 접근했을 경우의 한계점입니다.)

### 3) 추천시스템의 한계
 - Cold Start   
 Cold Start는 의미 그대로, '새로 시작할 때의 곤란함'을 의미합니다. 바로, 새로 생긴 매체, 장르, 콘텐츠에 대해서는 충분한 정보가 쌓이기 까지 추천대상에 언급될 수 없다는 것입니다. 새로운 대상을 소개하는 항목을 만들어서, 소비자가 직접 탐색하고 원하는 것을 찾아보고 고를 수 있도록 유도하는 부가적인 시스템이 필요할 것이라고 생각이 듭니다. 다행히도 이 문제는 콘텐츠를 기반으로 하는 필터링과, 모델 기반 필터링에서 다소 해결됩니다.

 - Filter bubble    
 얻는 것이 있으면 잃는 것이 있다고 하던가요? 추천 시스템은 방대한 정보의 홍수속에서 우리가 원하는 정보를 보다 쉽게 찾도록 만들어줍니다. 이러한 강점으로 상업적으로 전 분야에서는 추천 시스템이 점점 활성화되어가고 있습니다. 하지만 이런 환경 덕에 우리는 상품을 선택함에 있어 평소에 택하던 것과 다른 것을 둘러볼 기회를 다소 잃어버릴 가능성이 있습니다. 이러한 문제는 스스로의 노력으로 보다 다양한 사례를 접할 수 있도록 노력하는 것이 필요할 것입니다.     
 또 이런 문제점이 정치적 성향과 결합될 경우, 스스로의 노력으로도 극복되기 어려우며, 정치적 양극화를 초래할 위험이 크다는 문제가 있습니다. 이러한 문제를 잘 고려하여, 추천시스템을 개발할 필요가 있습니다.


<br><br><br><br>

- - - -
# 3. 코드 분석

## 0) 코드 흐름

### 먼저 데이터를 분석해서 알아보도록 하겠습니다. 오늘 활용할 데이터는 ml-1m으로 알려져있는 영화 평점 데이터입니다.
### 이어서 데이터 전처리로써 평점에 대한 CSR matrix를 만들어서 모델학습에 활용할 것입니다.

## 1) 데이터 준비

### 이제 영화데이터를 가져와, 추천시스템을 만들어보도록 하겠습니다.


In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
path = 'data/ml-1m/'
movie_col = ['movieID', 'Title', 'genre']
movie_data = pd.read_csv(path+'movies.dat',delimiter='::',encoding='ISO-8859-1',names=movie_col)

movie_data.head(10)



,movieID,Title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [2]:
rating_col = 'UserID::MovieID::Rating::Timestamp'.split('::')
rating_data = pd.read_csv(path+'ratings.dat',delimiter='::',encoding='ISO-8859-1',names=rating_col)

rating_data.head(10)

,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
5,1,1197,3,978302268
6,1,1287,5,978302039
7,1,2804,5,978300719
8,1,594,4,978302268
9,1,919,4,978301368


위에서 Timestamp는 소비자가 언제 영화를 봤는지를 알려줍니다. 한번 알기 쉬운 날짜로 바꿔볼까요?


In [3]:
import time
tm = time.gmtime(rating_data['Timestamp'][0])
print('{}년{}월{}일{}시{}분{}초{}번째요일{}번째날{}??'.format(*tm))
print(time.ctime(rating_data['Timestamp'][0]))
# dictt = {'dk':4, 'slj':26}
# print('{}{}'.format(*dictt))
# print('{dk}{slj}'.format(**dictt))

2000년12월31일22시12분40초6번째요일366번째날0??
Sun Dec 31 22:12:40 2000


### 이것으로 영화를 본 시간역시 추천 참고 데이터로 활용할 수 있겠습니다. 

In [4]:
user_col = 'UserID::Gender::Age::Occupation::Zip-code'.split('::')
user_data = pd.read_csv(path+'users.dat',delimiter='::',encoding='ISO-8859-1',names=user_col)
user_data.head(10)

,UserID,Gender,Age,Occupation,Zip-code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455
5,6,F,50,9,55117
6,7,M,35,1,06810
7,8,M,25,12,11413
8,9,M,25,17,61614
9,10,F,35,1,95370


### 유저마다 성별, 나이, 직업이 나누어져 있습니다. 이 역시 분류기준으로 사용할 수 있겠군요.

### 한번 유저의 수를 확인해보겠습니다.

In [5]:
print(f"등록된 유저 수: {user_data['UserID'].nunique()}, 평가한 유저 수 : {rating_data['UserID'].nunique()}")

등록된 유저 수: 6040, 평가한 유저 수 : 6040


### 다행히도 특별히 등록되지 않은 유저가 평가를 하거나 한 데이터는 없는 것 같습니다.

### 이번에는 인기있는 영화가 어떤영화인지 조금 알아보겠습니다. 평가는 배제하고 많이 본 영화를 기준으로 해도 괜찮겠다는 생각이 듭니다.


In [6]:
Movie_count = rating_data.groupby('MovieID')['UserID'].count()

# pop_movies30 = pd.DataFrame(columns=movie_data.columns)
# for i in Movie_count.sort_values(ascending=False).head(30).index:
#     # pd.concat(pop_movies30,movie_data[movie_data['movieID'] == i])
#     movienice = movie_data[movie_data['movieID'] == i]
#     pop_movies30 = pop_movies30.append(movienice)
# # list(Movie_count.sort_values(ascending=False).head(30).index)
# # pop_movies30
# pop_movies30

best_idx = Movie_count.sort_values(ascending=False).head(50).index # MovieID 가 인덱스도 되어있다.

pop_movies30 = [movie_data[movie_data['movieID']==i] for i in best_idx]
pop_movies30 = pd.concat(pop_movies30,axis=0)
pop_movies30

,movieID,Title,genre
2789,2858,American Beauty (1999),Comedy|Drama
257,260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Fantasy|Sci-Fi
1178,1196,Star Wars: Episode V - The Empire Strikes Back...,Action|Adventure|Drama|Sci-Fi|War
1192,1210,Star Wars: Episode VI - Return of the Jedi (1983),Action|Adventure|Romance|Sci-Fi|War
476,480,Jurassic Park (1993),Action|Adventure|Sci-Fi
1959,2028,Saving Private Ryan (1998),Action|Drama|War
585,589,Terminator 2: Judgment Day (1991),Action|Sci-Fi|Thriller
2502,2571,"Matrix, The (1999)",Action|Sci-Fi|Thriller
1250,1270,Back to the Future (1985),Comedy|Sci-Fi
589,593,"Silence of the Lambs, The (1991)",Drama|Thriller


In [7]:
# 영화의 총 개수와, 시청된 횟수를 구해보겠습니다.
Movie_count.describe()

count    3706.000000
mean      269.889099
std       384.047838
min         1.000000
25%        33.000000
50%       123.500000
75%       350.000000
max      3428.000000
Name: UserID, dtype: float64

In [8]:
movie_data["movieID"].nunique()

3883

### 첫번째, count수치가 영화의 개수와 일치해야하는데, 그렇지 않군요. 어떻게 된걸까요? 아마 평가되지 않은 영화가 있는 것 같습니다.

In [9]:
rating_data['MovieID'].nunique()

3706

### 평가되지 않은 영화들은 아마 그만큼 인기가 없었던 영화일 것입니다. 하지만, 사람들에게 드러나지 않고 지나간 숨겨진 명작일지도 모릅니다. 일단 항목을 유지하도록 해볼게요!

In [10]:
no_rate = set(movie_data['movieID']) - set(rating_data['MovieID'])
no_rate = pd.DataFrame(no_rate,columns=['MovieID'])
print(len(no_rate))
no_rate

no_rate = [movie_data[movie_data['movieID'] == i] for i in no_rate['MovieID']]

no_rate = pd.concat(no_rate,axis=0)

no_rate

177


,movieID,Title,genre
2495,2564,"Empty Mirror, The (1999)",Drama
3520,3589,"Kill, Baby... Kill! (Operazione Paura) (1966)",Horror
3011,3080,"Goodbye, 20th Century (Zbogum na dvadesetiot v...",Drama|Sci-Fi
1032,1045,Love Is All There Is (1996),Comedy|Drama
1518,1557,Squeeze (1996),Drama
...,...,...,...
2478,2547,Harvest (1998),Drama
1488,1524,"Turning, The (1992)",Drama
2990,3059,British Intelligence (1940),Drama
3513,3582,"Jails, Hospitals & Hip-Hop (2000)",Drama


### 177개의 평가되지 않은 영화입니다. 다른 사람이 많이 보지 않은 영화를 따로 검색할 수 있게 해도 좋겠다는 생각이 듭니다.

In [11]:
# 각 유저가 몇개의 영화를 보았는지 확인해보겠습니다.
# 첫번째는 유저 수
user_count = rating_data.groupby('UserID')['MovieID'].count()
user_count.describe()

count    6040.000000
mean      165.597517
std       192.747029
min        20.000000
25%        44.000000
50%        96.000000
75%       208.000000
max      2314.000000
Name: MovieID, dtype: float64

### 아래와 같이 임의의 추천받을 사람을 만들어 보았는데... 만들고 보니, 맞게 추천되었는지 확인할 길이 없군요. 다른 방식을 사용해봐야겠습니다.

In [12]:
import numpy as np
a = np.random.randint(0,3000,10)

# print(a)

your_favorite = [movie_data[movie_data['movieID'] == i] for i in a]

your_favorite = pd.concat(your_favorite,axis=0)

your_favorite

,movieID,Title,genre
2015,2084,Newsies (1992),Children's|Musical
2761,2830,Cabaret Balkan (Bure Baruta) (1998),Drama
2862,2931,Time of the Gypsies (Dom za vesanje) (1989),Drama
136,138,"Neon Bible, The (1995)",Drama
2334,2403,First Blood (1982),Action
700,709,Oliver & Company (1988),Animation|Children's
1244,1264,Diva (1981),Action|Drama|Mystery|Romance|Thriller
1544,1585,Love Serenade (1996),Comedy
2786,2855,Nightmares (1983),Horror


### 계속 출력을 하다보니, 영화의 인덱스와, MovieID가 비슷하면서도 숫자가 조금씩 차이가 납니다. 혹시 MovieID가 동일한 영화들이 있는걸까요?

In [13]:
movie_data.tail()

,movieID,Title,genre
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama
3882,3952,"Contender, The (2000)",Drama|Thriller


In [14]:
movie_data[movie_data.duplicated(subset='movieID')]

,movieID,Title,genre


### 중복되는 숫자는 없고, 하나씩 건너뛰는 숫자가 있는 것 같아요;;
## 2) 추천 시스템 만들기

### 자 이제 추천시스템을 위한 고객을 만들어보겠습니다!! 먼저 대상고객이 이름을 정하고, 고객이 봤던 영화들로 평가를 몇개만 하도록 해볼까요? 이왕이면, 정말의 의미있는 추천이 나오는지, 제가 봤던 영화 몇가지를 넣어보도록 할게요! 고객명도 hchang으로 해보겠습니다.


### 영화를 위의 movies.dat파일로 하나하나 넘기면서 찾다보니, 영화가 매우 많아서 찾기 어렵다는걸 깨달았습니다ㅜㅜ 추천을 위해서는 고객의 평가자료가 필수적으로 있어야할텐데..

In [15]:
import re
import numpy as np



year_set = set(movie_data['Title'].apply(lambda x : re.findall("([0-9]+)",x)[-1]))

np.array(list(year_set))
year_ar = np.array(sorted([int(i) for i in year_set]))
year_ar

array([1919, 1920, 1921, 1922, 1923, 1925, 1926, 1927, 1928, 1929, 1930,
       1931, 1932, 1933, 1934, 1935, 1936, 1937, 1938, 1939, 1940, 1941,
       1942, 1943, 1944, 1945, 1946, 1947, 1948, 1949, 1950, 1951, 1952,
       1953, 1954, 1955, 1956, 1957, 1958, 1959, 1960, 1961, 1962, 1963,
       1964, 1965, 1966, 1967, 1968, 1969, 1970, 1971, 1972, 1973, 1974,
       1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985,
       1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996,
       1997, 1998, 1999, 2000])

### 일단 모두 2000년 이전의 영화였군요.. 가장 인기가 많았던 영화에서 찾아보겠습니다.

### 제가 봤던영화 평가는 다음과 같습니다.

Toy Story (1995)  MovieID = 1 4점      

Terminator 2: Judgment Day (1991)   MovieID = 589 4점       

Matrix, The (1999)	MovieID = 2571 5점      

Sixth Sense, The (1999)	MovieID = 2762 5점      

Schindler's List (1993)	MovieID = 527 2점       

In [33]:
# rating_data의 columns:  UserID	MovieID	Rating	Timestamp
import time
my_list = ['Toy Story (1995)', 'Terminator 2: Judgment Day (1991)', 'Matrix, The (1999)', 'Sixth Sense, The (1999)', "Schindler's List (1993)"]
my_list_num = [movie_data[movie_data['Title'] == i]["movieID"].values[0] for i in my_list]
my_rating = [4,4,5,5,2]
my_rating_data = pd.DataFrame({'UserID': ['hchang']*5, 'MovieID':my_list_num, 'Rating':my_rating, 'Timestamp':[int(time.time())]*5})

if not rating_data.isin({'UserID':['hchang']})['UserID'].any():
    addition_data = rating_data.append(my_rating_data, ignore_index=True)

# timestamp는 이번 프로젝트에서는 중요치 않으므로 넘어가도록 하겠습니다.
# 다음에는 고려해보도록 할게요!

addition_data.tail(10)

,UserID,MovieID,Rating,Timestamp
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648
1000208,6040,1097,4,956715569
1000209,hchang,1,4,1636913913
1000210,hchang,589,4,1636913913
1000211,hchang,2571,5,1636913913
1000212,hchang,2762,5,1636913913
1000213,hchang,527,2,1636913913


In [34]:
# 위에서 봤다시피, movieID에는 사이사이에, 넘어가는 숫자가 있을 것입니다.
# Matrix Fatorization을 만드는데, 방해가 될 요소이므로
# 이제 movieID와, UserID를 다시 인덱싱하겠습니다.

user_unique = addition_data['UserID'].unique()
movie_unique = movie_data['movieID'].unique()

user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}


In [35]:
print(user_to_idx['hchang'])
print(movie_to_idx[2557])

6040
2488


In [36]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
# temp_user_data = data['user_id'].map(user_to_idx.get).dropna()
if len(user_data)+1 == len(user_to_idx):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    # data['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print(len(user_data),len(user_to_idx))
    print('user_id column indexing Fail!!')

# artist_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
# temp_artist_data = data['artist'].map(artist_to_idx.get).dropna()
if len(movie_data) == len(movie_to_idx):
    print('artist column indexing OK!!')
    # data['artist'] = temp_artist_data
else:
    print('artist column indexing Fail!!')


user_id column indexing OK!!
artist column indexing OK!!


### 이제 위에서 설명한 csr matrix를 만들어보려 합니다. 여기서, 평가 점수를 무엇으로 잡을지 잘 생각해야합니다.
### 여기서는 당연히 평가 점수를 그대로 사용해보겠습니다.

In [37]:
# 이제 csr_matrix를 만들어보겠습니다.
from scipy.sparse import csr_matrix

num_user = len(user_to_idx)
num_movie = len(movie_to_idx)

user_idx = addition_data['UserID']
movie_idx = addition_data["MovieID"]
user_idx = [user_to_idx[i] for i in user_idx]
movie_idx = [movie_to_idx[i] for i in movie_idx]
csr_data = csr_matrix((addition_data['Rating'], (user_idx, movie_idx)), shape= (num_user, num_movie))
csr_data

<6041x3883 sparse matrix of type '<class 'numpy.longlong'>'
	with 1000214 stored elements in Compressed Sparse Row format>

In [117]:
# itir = csr_data.__iter__()
next(itir)

<1x3883 sparse matrix of type '<class 'numpy.longlong'>'
	with 198 stored elements in Compressed Sparse Row format>

### 위를 보시면, AIFFEL의 권고사항을 하나 무시했습니다. 그 내용은 바로 아래 내용입니다.
 - 별점을 시청횟수로 해석해서 생각하겠습니다.
 - 또한 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외하겠습니다.
<br><br>    

### 엄격하게 봤을때, 위의 가정은 적절하지 않습니다. 왜냐하면, 시청횟수는 저희가 고객의 평가에 대한 해석을 보다 요구했던 반면에, 현재 평점 데이터는 그렇지 않습니다. 그저 그대로 사용하였을때 더 좋은 결과를 가져올 수 있습니다. 

### 또 3점 이하의 데이터는 선호하지 않는 것으로 가정하고 제외하라고 했는데, 제외할 경우는 선호하지 않는 것이 아니라, 반영이 되지 않습니다. 고객은 싫다고 별점을 1개로 반영했는데, 학습에서 제외할 경우, 시스템은 고객이 이 영화를 좋아할 수도 있다고 판단 할 수 있습니다. 따라서 해당 평점은 모두 그대로 반영하는 것으로 학습하겠습니다.

In [38]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [44]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [47]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3883x6041 sparse matrix of type '<class 'numpy.longlong'>'
	with 1000214 stored elements in Compressed Sparse Column format>

In [48]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

### 기나긴 전처리에 비하면, 학습이 매우 빨리 끝났습니다!    
### 이제 모델을 평가해보도록 하겠습니다.

- - - - 

## 3) 모델 평가

### 먼저, 저희는 MovieID를 index로 바꿔주어 Matrix를 만들어 사용했습니다. 하지만, 영화의 이름을 확인해야 의미가 있겠죠?

In [61]:
def movie_to_movieID(movie):
    return movie_data[movie_data['Title']==movie]['movieID'].values[0]

def movieID_to_movie(movieID):
    return movie_data[movie_data['movieID']==movieID]['Title'].values[0]

a= movie_to_movieID("Schindler's List (1993)")
a, movieID_to_movie(a)


(527, "Schindler's List (1993)")

In [52]:
hchang, Schindler = user_to_idx['hchang'], movie_to_idx[movie_to_movieID("Schindler's List (1993)")]
hchang_vector, Schindler_vector = als_model.user_factors[hchang], als_model.item_factors[Schindler]

print('슝=3')

슝=3


In [53]:
hchang_vector, Schindler_vector

(array([-0.49659234, -0.13967298,  0.50988114, -0.02727161,  0.47986805,
        -0.17533647, -0.32223257, -0.49286333, -0.79650396,  0.12790217,
        -0.36498567,  0.6386416 , -0.01528621,  1.0378968 ,  0.22040978,
        -0.34891674,  0.14843298, -0.6927679 ,  0.00333155,  0.2662033 ,
         0.31352833,  0.06079116,  0.7835109 , -0.06796301, -0.42841268,
        -0.18571556,  0.22712193, -0.24973749,  0.77665246, -0.2502074 ,
         0.61551666,  0.57294065,  0.39552292,  0.08009509, -0.570386  ,
        -0.06643521, -0.56513435,  0.00989503, -0.42572564, -0.3993912 ,
        -0.34559834, -0.05187041,  0.76233906, -0.43973866, -0.4315041 ,
        -1.1945319 , -0.33607566,  0.253948  ,  0.23201506, -0.03014215,
         0.8563968 ,  0.48289347,  0.35567236,  0.991812  , -0.12539734,
         0.38147807,  0.18486038, -0.28780085,  0.45522317, -0.169937  ,
         0.07935619,  0.33941188, -0.36694854,  0.14441778, -0.32424948,
         0.13274373,  0.10772956,  0.7264519 , -0.1

In [54]:
np.dot(hchang_vector, Schindler_vector)

0.3137865

In [58]:
Matrix = movie_to_idx[movie_to_movieID("Matrix, The (1999)")]
Matrix_vector = als_model.item_factors[Matrix]

np.dot(hchang_vector, Matrix_vector)

0.5567541

### 이와같이, 제가 비교적 별로라고 평가한 영화는 점수가 낮고, 좋다고 한 영화는 수치가 보다 높게 나오고 있습니다. 지금까지는 매우 마음에 듭니다 :)

In [73]:
favorite_movie = 'Matrix, The (1999)'
favorite_movieID = movie_to_movieID(favorite_movie)
movie_idx = movie_to_idx[favorite_movieID]
similar_movie = als_model.similar_items(movie_idx, N=15)
similar_movie

[(2502, 1.0000001),
 (585, 0.81919825),
 (2847, 0.70709324),
 (453, 0.66839796),
 (1220, 0.57435185),
 (476, 0.5707613),
 (1533, 0.56568235),
 (1491, 0.5315002),
 (1539, 0.5287283),
 (1568, 0.5071035),
 (257, 0.43992546),
 (373, 0.41592047),
 (108, 0.41580468),
 (31, 0.4060399),
 (1178, 0.40216056)]

In [78]:
movieidx_to_movieID = {k:v for v,k in movie_to_idx.items()}


In [79]:
for i in similar_movie:
    movieID = movieidx_to_movieID[i[0]]
    print(movieID_to_movie(movieID))


Matrix, The (1999)
Terminator 2: Judgment Day (1991)
Total Recall (1990)
Fugitive, The (1993)
Terminator, The (1984)
Jurassic Park (1993)
Face/Off (1997)
Fifth Element, The (1997)
Men in Black (1997)
Hunt for Red October, The (1990)
Star Wars: Episode IV - A New Hope (1977)
Speed (1994)
Braveheart (1995)
Twelve Monkeys (1995)
Star Wars: Episode V - The Empire Strikes Back (1980)


### 영화 제목 때문에 혼동이 있지는 않으시죠?ㅎㅎ
### 영화 Matrix와 비슷한 영화도 올바르게 추천해주고 있는 것으로 보입니다.

### 그러면 저에게 어떤 영화를 추천해주는지 확인해봅시다.

In [80]:
user = user_to_idx['hchang']
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
# movie_recommended
for i in movie_recommended:
    movieID = movieidx_to_movieID[i[0]]
    print(movieID_to_movie(movieID))

Silence of the Lambs, The (1991)
Fugitive, The (1993)
Toy Story 2 (1999)
Saving Private Ryan (1998)
Jurassic Park (1993)
Braveheart (1995)
Shawshank Redemption, The (1994)
Star Wars: Episode I - The Phantom Menace (1999)
Forrest Gump (1994)
Usual Suspects, The (1995)
Fargo (1996)
Terminator, The (1984)
Groundhog Day (1993)
Total Recall (1990)
L.A. Confidential (1997)
American Beauty (1999)
Hunt for Red October, The (1990)
Men in Black (1997)
Back to the Future (1985)
Star Wars: Episode IV - A New Hope (1977)


### 제가 만든 모델이 잘 추천해줬는지 확인을 해야하니, 주말에 영화를 몰아서 봐야겠군요ㅎㅎ
### 위를 보면 Toy Story 2 (1999)가 눈에 띄는군요? 비슷한 영화인 것을 알고 추천해준 것이 대단합니다. 한번 왜 저 영화를 추천해줬는지 확인해 보겠습니다!!

In [81]:
Toy2 = movie_to_idx[movie_to_movieID('Toy Story 2 (1999)')]
Toy2

explain = als_model.explain(user, csr_data, itemid=Toy2)


[(movieID_to_movie(movieidx_to_movieID[i[0]]),i[1]) for i in explain[1]]

[('Toy Story (1995)', 0.36030381044425486),
 ('Sixth Sense, The (1999)', 0.045207494873799325),
 ("Schindler's List (1993)", 0.02320640869542556),
 ('Terminator 2: Judgment Day (1991)', -0.003690057926850509),
 ('Matrix, The (1999)', -0.010489815060869455)]

### 놀랍게도, Toy Story 1편을 본 것을 보고, 추천해줬군요!! 너무 신기합니다.ㅎㅎ

# 4. 아쉬운 점

### 1 비교적 매우 과거의 데이터로만 많이 아쉬운 느낌이 없을 수가 없군요.

### 2 시기에 따른 추천을 어떻게 반영할 수 있을까 궁금합니다.

### 3 분명 관심은 있었는데, 전처리 과정에서, 생각처럼 머리가 따라주지 않아서... 전처리의 중요성을 다시한번 깨달았습니다. 사실 보면 별로 한 것도 없는데, 이걸 얼마나 한건지ㅜㅜ

### 4 csr Matrix의 구조는 익혔지만, 파이썬에서 csrMatrix가 어떻게 구동되고 있는지 아직 잘 와닫지가 않습니다. 조금 더 다뤄봐야할 것 같습니다!